In [1]:
import xlrd 
import pubchempy as pcp
import pickle

import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
import seaborn as sns

from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec, mol2sentence
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg

In [2]:
pickle_in = open("fold-0.pickle","rb")
load_dict =  pickle.load(pickle_in)

data_frame = load_dict['shuffled']

data_frame

,No.,FileHandle,SoluteName,Formula,Subset,Charge,Level1,Level2,Level3,Solvent,...,HP,OP,S,HS,OS,SP,SS,TotalArea,SoluteSMILES,SolventSMILES
1758,1759,0438pho,"diethyl2,4-dichlorophenylthiophosphate",H13C10O3P1S1CL2,[g],0,11,4,0,octanol,...,0.000,16.033,34.582,0.0,0.0,24.737,0.0,331.933,C1C2C(C(C1Cl)Cl)C3(C(=C(C2(C3(Cl)Cl)Cl)Cl)Cl)Cl,CCCCCCCCO
788,789,0071proa,propanal,H6C3O1,[g],0,2,4,1,diethylether,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,124.289,CCC=O,CCOCC
1726,1727,0210dic,"1,1-dichloro-2,2-difluoroethylmethylether",H4C3O1F2CL2,[g],0,9,1,0,octanol,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,178.993,CC1=CC(=O)CC(C1)(C)C,CCCCCCCCO
1939,1940,0110but,butylamine,H11C4N1,[g],0,3,1,0,secbutanol,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,164.243,CCCCN,CCC(C)O
1467,1468,0008noc,n-octane,H18C8,[g],0,2,1,1,methylformamide,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,243.846,CCCCCCCC,CNC=O
2110,2111,0021eth,ethene,H4C2,[a],0,2,1,4,water,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,87.417,C=C,O
739,740,0087pro,propanoicacid,H6C3O2,[g],0,2,5,0,dichloroethane,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,135.630,CCC(=O)O,CC(Cl)Cl
1196,1197,0172Edi,"E-1,2-dichloroethene",H2C2CL2,[g],0,6,2,0,hexadecane,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,126.855,C(=CCl)Cl,CCCCCCCCCCCCCCCC
111,112,0150mhy,m-hydroxybenzaldehyde,H6C7O2,[g],0,2,9,0,benzene,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,179.066,C1=CC(=CC(=C1)O)C=O,C1=CC=CC=C1
1820,1821,test4002,diiodomethane,H2C1I2,[g],0,15,0,0,octanol,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,136.531,C(I)I,CCCCCCCCO


In [3]:
# loc = ("MNSol_alldata.xlsx") 
# wb = xlrd.open_workbook(loc) 
# sheet = wb.sheet_by_index(0) 

# solutes = sheet.col_values(2)
# solvents = sheet.col_values(9)
# energies = sheet.col_values(10)

# solute_smis = []
# solvent_smis = []

# for solute in solutes[100:110]:
#     for compound in pcp.get_compounds(solute, 'name'):
#         solute_smis.append(compound.isomeric_smiles)
        
# for solvent in solvents[100:110]:
#     for compound in pcp.get_compounds(solvent, 'name'):
#         solvent_smis.append(compound.isomeric_smiles)

solute_smis = data_frame['SoluteSMILES'].tolist()
solvent_smis = data_frame['SolventSMILES'].tolist()
energies = data_frame['DeltaGsolv'].tolist()

print(len(solute_smis))
print(len(solvent_smis))
print(len(energies))

2530
2530
2530


In [4]:
solutesmis = [Chem.MolFromSmiles(x) for x in solute_smis]
solventsmis = [Chem.MolFromSmiles(x) for x in solvent_smis]

RDKit WARNING: [05:40:07] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [05:40:07] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [05:40:07] WARNING: not removing hydrogen atom without neighbors


In [5]:
solute_sentences = []
solvent_sentences = []

for solute_smi in solutesmis:
    solute_sentences.append(mol2alt_sentence(solute_smi, 1))
    
for solvent_smi in solventsmis:
    solvent_sentences.append(mol2alt_sentence(solvent_smi, 1))
    
print(solute_sentences[6])
print(solvent_sentences[6])

['2246728737', '3542456614', '2245384272', '2064788354', '2246699815', '4278941385', '864942730', '1510328189', '864662311', '1533864325']
['2246728737', '3537119515', '2245273601', '2406620359', '1016841875', '170898322', '1016841875', '170898322']


In [6]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('model_300dim.pkl')

solute_sequences = []
count = 0
sequences = []
solvent_sequences = []
Gsolv = []
l = len(solute_smis)

for i in range(l):
    flag = 0
    solute_substructures = []
    solvent_substructures = []
    
    for identifier in solute_sentences[i]:
        try:
            solute_substructures.append(model.wv.word_vec(identifier))
        except:
            flag = 1
            break
    
    for identifier in solvent_sentences[i]:
        try:
            solvent_substructures.append(model.wv.word_vec(identifier))
        except:
            flag = 1
            break
            
    if flag == 1:
        count += 1
        continue
    sequences.append((solute_substructures, solvent_substructures))
    solute_sequences.append(solute_substructures)
    solvent_sequences.append(solvent_substructures)
    Gsolv.append(energies[i])
    
    
print(count)
# solute_mol2vecs = [DfVec(x) for x in sentences2vec(solute_sentences, model, unseen='UNK')]
# solvent_mol2vecs = [DfVec(x) for x in sentences2vec(solvent_sentences, model, unseen='UNK')]

401


In [7]:
print(len(solute_sequences))
print(len(solvent_sequences))
print(len(Gsolv))
print(len(sequences))

# for i in solute_sequences:
#     print(np.asarray(i).shape)
# print('\n')
# for i in solvent_sequences:
#     print(np.asarray(i).shape)

2129
2129
2129
2129


In [8]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

In [9]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layer, output_dim, batch_size, bidirectional = True):
        super(LSTMModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layer = num_layer
        self.bidirectional = bidirectional
        self.batch_size = batch_size

        self.solv_lstm = nn.LSTM(input_dim, hidden_dim, num_layer, batch_first=True, bidirectional = bidirectional)
        self.solu_lstm = nn.LSTM(input_dim, hidden_dim, num_layer, batch_first=True, bidirectional = bidirectional)

        self.fc1 = nn.Linear(hidden_dim*2*2, 2000)
        self.fc2 = nn.Linear(2000, output_dim)
        
    def forward(self, inputs_solv, inputs_solu):
        # Initialize hidden state
        h0_solv = torch.zeros(self.num_layer*(1 + int(self.bidirectional)), self.batch_size, self.hidden_dim)
        h0_solu = torch.zeros(self.num_layer*(1 + int(self.bidirectional)), self.batch_size, self.hidden_dim)

        # Initialize cell state
        c0_solv = torch.zeros(self.num_layer*(1 + int(self.bidirectional)), self.batch_size, self.hidden_dim)
        c0_solu = torch.zeros(self.num_layer*(1 + int(self.bidirectional)), self.batch_size, self.hidden_dim)
        
        H, _ = self.solv_lstm(inputs_solv, (h0_solv, c0_solv)) # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        G, _ = self.solu_lstm(inputs_solu, (h0_solu, c0_solu))
        
        # shared attention layer
        Hsize = H.size(1)
        Gsize = G.size(1)
        a_score = torch.zeros(Hsize, Gsize)
        
        for i in range(Hsize):
            for j in range(Gsize):
                a_score[i][j] = torch.matmul(H[0][i],torch.t(G[0][j]))
                
        
        a = F.softmax(a_score,1)
        
        G = torch.squeeze(G, axis=0)
        H = torch.squeeze(H, axis=0)
        
        P = torch.matmul(a, G)
        Q = torch.matmul(torch.t(a), H)
        
        # maxpooling layer
        u = torch.max(H, P)
        v = torch.max(G, Q)
        
        inpu = torch.sum(u, dim=0)
        inpv = torch.sum(v, dim=0)
        
        inp = torch.cat((inpu, inpv), 0)
        
        # mlp
        x = F.relu(self.fc1(inp)) #number of layers? concat? sum?
        solvE = self.fc2(x)
      
        return solvE


In [10]:
bidirectional = True

input_dim = 300
hidden_dim = 150
layer_dim = 1
output_dim = 1

batch_size = 1

In [14]:
model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim, batch_size)

# PRINTING MODEL & PARAMETERS 
# print(model)
print(len(list(model.parameters())))
# print(list(model.parameters()))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())
    
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

20
torch.Size([600, 300])
torch.Size([600, 150])
torch.Size([600])
torch.Size([600])
torch.Size([600, 300])
torch.Size([600, 150])
torch.Size([600])
torch.Size([600])
torch.Size([600, 300])
torch.Size([600, 150])
torch.Size([600])
torch.Size([600])
torch.Size([600, 300])
torch.Size([600, 150])
torch.Size([600])
torch.Size([600])
torch.Size([2000, 600])
torch.Size([2000])
torch.Size([1, 2000])
torch.Size([1])
The model has 2,288,801 trainable parameters


In [21]:
learning_rate = 0.0002
nest_momentum = 0.9

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=nest_momentum, nesterov=True)

TRAIN: 1705 VALIDATION: 211 TEST: 213
211
[ -3.57  -5.76  -5.66  -5.83  -7.26 -11.55  -5.43  -5.51  -3.13  -2.88
  -6.11  -5.81  -4.15  -7.29  -5.03  -4.99  -7.86  -8.75  -5.68  -4.19
  -3.55  -5.89  -3.42  -1.53  -2.96  -6.62  -3.56  -9.38  -6.05  -5.62
  -2.67  -4.86  -4.42 -16.92  -1.29  -8.84  -5.65  -4.62  -5.12  -6.5
  -6.32  -1.66  -3.47  -4.62  -4.64  -2.06  -2.1  -11.35  -5.52  -4.16
  -9.22  -3.83  -6.16  -0.87 -11.61  -1.73  -7.1  -11.36  -5.49  -3.29
  -4.14  -8.58  -7.54  -5.33  -6.35  -4.56  -5.15  -7.34  -4.27  -3.59
  -3.59  -3.9   -4.38  -2.39  -6.37  -6.33  -7.23  -4.25 -12.18  -5.98
  -5.02  -4.9   -2.15  -4.74  -4.02  -6.35  -4.43  -3.69  -2.86  -4.41
  -5.54  -2.79  -6.01  -4.09  -6.67  -4.45  -6.83  -9.63  -7.79  -4.1
  -4.92  -3.7   -6.94  -3.45  -5.14  -5.29  -7.4   -6.7   -4.73  -5.98
  -3.6   -6.21  -4.81 -11.56  -5.47  -6.17  -5.01  -0.39  -6.19  -5.01
  -7.77  -3.82  -6.94  -7.11  -6.67  -5.05  -6.77  -4.52  -5.51  -0.12
  -4.93  -2.37  -4.98  -6.66  -7.62  

In [31]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

num_epochs = 100
splits = 10

best_model = model
best_loss = 10000
losses = []

# Training and 10 fold cross validation
kf = KFold(n_splits = 10)
kf.get_n_splits(sequences)

for train_index, test_index in kf.split(sequences):
    model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim, batch_size)
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=nest_momentum, nesterov=True)
    
    seq_tr_va, seq_test = np.asarray(sequences)[train_index], np.asarray(sequences)[test_index]
    E_tr_va, E_test = np.asarray(Gsolv)[train_index], np.asarray(Gsolv)[test_index]
    seq_train, seq_val, E_train, E_val = train_test_split(seq_tr_va, E_tr_va, test_size=0.11)
    
    print("TRAIN:", len(seq_train), "VALIDATION:", len(seq_val), "TEST:", len(seq_test))
    
    #Training
    for epoch in range(num_epochs):       
        for i in range(len(seq_train)):
            solute, solvent = seq_train[i]
            energy = torch.FloatTensor([E_train[i]])
        
            # Forward pass
            output = model((torch.FloatTensor([solvent])), (torch.FloatTensor([solute])))
            loss = criterion(output, energy)
        
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
            
        #Validation
        outputs = []

        with torch.no_grad():
            for i in range(len(seq_val)):
                solute, solvent = seq_val[i]
                energy = torch.FloatTensor([E_val[i]])
        
                output = model((torch.FloatTensor([solvent])), (torch.FloatTensor([solute])))
                outputs.append(output)
                
            val_loss = criterion(torch.squeeze(torch.FloatTensor(E_val), axis=0), torch.squeeze(torch.FloatTensor([outputs]), axis=0))
            print('Validation MSE: {}'.format(val_loss))
            if val_loss < best_loss:
                best_loss = val_loss
                best_model = model
            
    #Testing
    outputs = []

    with torch.no_grad():
        for i in range(len(seq_test)):
            solute, solvent = seq_test[i]
            energy = torch.FloatTensor([E_test[i]])
        
            output = best_model((torch.FloatTensor([solvent])), (torch.FloatTensor([solute])))
            outputs.append(output)
                
        test_loss = criterion(torch.squeeze(torch.FloatTensor(E_test), axis=0), torch.squeeze(torch.FloatTensor([outputs]), axis=0))
        print('Test MSE: {}'.format(test_loss))
        losses.append(test_loss)
    

TRAIN: 1705 VALIDATION: 211 TEST: 213
Epoch [1/100], Loss: 18.8111
Epoch [1/100], Loss: 5.6896
Epoch [1/100], Loss: 23.2495
Epoch [1/100], Loss: 9.9794
Epoch [1/100], Loss: 74.8494
Epoch [1/100], Loss: 26.9552
Epoch [1/100], Loss: 30.3662
Epoch [1/100], Loss: 2.3268
Epoch [1/100], Loss: 0.0255
Epoch [1/100], Loss: 40.4690
Epoch [1/100], Loss: 4.3423
Epoch [1/100], Loss: 2.1925
Epoch [1/100], Loss: 0.3941
Epoch [1/100], Loss: 1.2757
Epoch [1/100], Loss: 14.2375
Epoch [1/100], Loss: 0.0420
Epoch [1/100], Loss: 1.6360
Epoch [1/100], Loss: 12.3818
Epoch [1/100], Loss: 0.5505
Epoch [1/100], Loss: 0.9461
Epoch [1/100], Loss: 0.7565
Epoch [1/100], Loss: 0.4014
Epoch [1/100], Loss: 0.4294
Epoch [1/100], Loss: 6.2570
Epoch [1/100], Loss: 0.2270
Epoch [1/100], Loss: 5.9103
Epoch [1/100], Loss: 0.0019
Epoch [1/100], Loss: 16.7849
Epoch [1/100], Loss: 3.3251
Epoch [1/100], Loss: 1.8748
Epoch [1/100], Loss: 4.6551
Epoch [1/100], Loss: 0.1331
Epoch [1/100], Loss: 2.6146
Epoch [1/100], Loss: 0.0207
E

Epoch [1/100], Loss: 1.1660
Epoch [1/100], Loss: 13.7426
Epoch [1/100], Loss: 1.6004
Epoch [1/100], Loss: 1.4501
Epoch [1/100], Loss: 1.2089
Epoch [1/100], Loss: 0.0398
Epoch [1/100], Loss: 1.5652
Epoch [1/100], Loss: 1.4915
Epoch [1/100], Loss: 1.6478
Epoch [1/100], Loss: 0.9445
Epoch [1/100], Loss: 6.3704
Epoch [1/100], Loss: 14.9835
Epoch [1/100], Loss: 9.2885
Epoch [1/100], Loss: 0.0828
Epoch [1/100], Loss: 22.1221
Epoch [1/100], Loss: 12.1947
Epoch [1/100], Loss: 23.7701
Epoch [1/100], Loss: 10.8133
Epoch [1/100], Loss: 0.2689
Epoch [1/100], Loss: 15.8349
Epoch [1/100], Loss: 0.0302
Epoch [1/100], Loss: 11.8935
Epoch [1/100], Loss: 15.1619
Epoch [1/100], Loss: 9.8710
Epoch [1/100], Loss: 3.1591
Epoch [1/100], Loss: 3.0967
Epoch [1/100], Loss: 27.5556
Epoch [1/100], Loss: 1.4491
Epoch [1/100], Loss: 0.0991
Epoch [1/100], Loss: 3.0256
Epoch [1/100], Loss: 0.0000
Epoch [1/100], Loss: 0.3120
Epoch [1/100], Loss: 0.5371
Epoch [1/100], Loss: 6.7837
Epoch [1/100], Loss: 14.4742
Epoch [1/

Epoch [1/100], Loss: 0.8799
Epoch [1/100], Loss: 1.6127
Epoch [1/100], Loss: 10.3532
Epoch [1/100], Loss: 25.5801
Epoch [1/100], Loss: 0.0058
Epoch [1/100], Loss: 1.9979
Epoch [1/100], Loss: 62.3576
Epoch [1/100], Loss: 3.3428
Epoch [1/100], Loss: 2.5719
Epoch [1/100], Loss: 110.1171
Epoch [1/100], Loss: 0.0104
Epoch [1/100], Loss: 1.5855
Epoch [1/100], Loss: 2.3858
Epoch [1/100], Loss: 1.7123
Epoch [1/100], Loss: 0.9523
Epoch [1/100], Loss: 2.8212
Epoch [1/100], Loss: 0.4585
Epoch [1/100], Loss: 10.8138
Epoch [1/100], Loss: 4.5863
Epoch [1/100], Loss: 0.4428
Epoch [1/100], Loss: 0.7640
Epoch [1/100], Loss: 1.5541
Epoch [1/100], Loss: 0.3445
Epoch [1/100], Loss: 0.2942
Epoch [1/100], Loss: 8.8981
Epoch [1/100], Loss: 1.3055
Epoch [1/100], Loss: 0.0176
Epoch [1/100], Loss: 0.3120
Epoch [1/100], Loss: 6.5240
Epoch [1/100], Loss: 0.0281
Epoch [1/100], Loss: 0.0018
Epoch [1/100], Loss: 1.8487
Epoch [1/100], Loss: 19.8217
Epoch [1/100], Loss: 0.3138
Epoch [1/100], Loss: 0.5620
Epoch [1/100]

Epoch [1/100], Loss: 9.5582
Epoch [1/100], Loss: 0.3033
Epoch [1/100], Loss: 0.0139
Epoch [1/100], Loss: 1.0471
Epoch [1/100], Loss: 1.1106
Epoch [1/100], Loss: 0.4100
Epoch [1/100], Loss: 0.5734
Epoch [1/100], Loss: 3.1317
Epoch [1/100], Loss: 8.9142
Epoch [1/100], Loss: 1.9813
Epoch [1/100], Loss: 35.0809
Epoch [1/100], Loss: 0.4874
Epoch [1/100], Loss: 0.0188
Epoch [1/100], Loss: 0.1036
Epoch [1/100], Loss: 8.5783
Epoch [1/100], Loss: 10.7388
Epoch [1/100], Loss: 1.6033
Epoch [1/100], Loss: 4.8483
Epoch [1/100], Loss: 0.6341
Epoch [1/100], Loss: 0.6424
Epoch [1/100], Loss: 0.5439
Epoch [1/100], Loss: 23.3186
Epoch [1/100], Loss: 64.5281
Epoch [1/100], Loss: 2.5291
Epoch [1/100], Loss: 26.1973
Epoch [1/100], Loss: 0.4174
Epoch [1/100], Loss: 0.1992
Epoch [1/100], Loss: 0.5293
Epoch [1/100], Loss: 10.4578
Epoch [1/100], Loss: 24.3138
Epoch [1/100], Loss: 0.0004
Epoch [1/100], Loss: 0.0024
Epoch [1/100], Loss: 193.0038
Epoch [1/100], Loss: 177.2370
Epoch [1/100], Loss: 239.6032
Epoch [

Epoch [1/100], Loss: 0.0001
Epoch [1/100], Loss: 13.6197
Epoch [1/100], Loss: 27.8204
Epoch [1/100], Loss: 1.6469
Epoch [1/100], Loss: 3.7170
Epoch [1/100], Loss: 6.5152
Epoch [1/100], Loss: 0.8383
Epoch [1/100], Loss: 9.4853
Epoch [1/100], Loss: 11.2253
Epoch [1/100], Loss: 4.5419
Epoch [1/100], Loss: 1.7570
Epoch [1/100], Loss: 0.4917
Epoch [1/100], Loss: 0.1638
Epoch [1/100], Loss: 0.5765
Epoch [1/100], Loss: 4.2940
Epoch [1/100], Loss: 5.8716
Epoch [1/100], Loss: 0.4456
Epoch [1/100], Loss: 0.1190
Epoch [1/100], Loss: 1.3201
Epoch [1/100], Loss: 0.6219
Epoch [1/100], Loss: 0.7795
Epoch [1/100], Loss: 1.3577
Epoch [1/100], Loss: 0.4744
Epoch [1/100], Loss: 0.1624
Epoch [1/100], Loss: 0.6159
Epoch [1/100], Loss: 0.7824
Epoch [1/100], Loss: 0.0826
Epoch [1/100], Loss: 0.0133
Epoch [1/100], Loss: 17.6971
Epoch [1/100], Loss: 1.2963
Epoch [1/100], Loss: 1.5566
Epoch [1/100], Loss: 2.6565
Epoch [1/100], Loss: 5.2589
Epoch [1/100], Loss: 1.3671
Epoch [1/100], Loss: 0.5622
Epoch [1/100], L

...

Epoch [23/100], Loss: 0.0045
Epoch [23/100], Loss: 0.0182
Epoch [23/100], Loss: 0.2073
Epoch [23/100], Loss: 0.2536
Epoch [23/100], Loss: 0.0001
Epoch [23/100], Loss: 0.0209
Epoch [23/100], Loss: 0.0373
Epoch [23/100], Loss: 0.2685
Epoch [23/100], Loss: 0.0919
Epoch [23/100], Loss: 0.0034
Epoch [23/100], Loss: 0.0325
Epoch [23/100], Loss: 0.3942
Epoch [23/100], Loss: 0.0283
Epoch [23/100], Loss: 0.0133
Epoch [23/100], Loss: 0.0911
Epoch [23/100], Loss: 0.0053
Epoch [23/100], Loss: 0.2650
Epoch [23/100], Loss: 0.0626
Epoch [23/100], Loss: 1.1732
Epoch [23/100], Loss: 0.1145
Epoch [23/100], Loss: 0.0249
Epoch [23/100], Loss: 0.0398
Epoch [23/100], Loss: 0.1248
Epoch [23/100], Loss: 0.1326
Epoch [23/100], Loss: 0.3264
Epoch [23/100], Loss: 8.1936
Epoch [23/100], Loss: 0.2000
Epoch [23/100], Loss: 0.5168
Epoch [23/100], Loss: 0.0006
Epoch [23/100], Loss: 0.0002
Epoch [23/100], Loss: 0.3276
Epoch [23/100], Loss: 0.1515
Epoch [23/100], Loss: 0.2522
Epoch [23/100], Loss: 0.5724
Epoch [23/100]

Epoch [23/100], Loss: 0.0000
Epoch [23/100], Loss: 0.2024
Epoch [23/100], Loss: 0.0045
Epoch [23/100], Loss: 0.0336
Epoch [23/100], Loss: 0.0320
Epoch [23/100], Loss: 1.1627
Epoch [23/100], Loss: 0.0289
Epoch [23/100], Loss: 0.0111
Epoch [23/100], Loss: 0.0384
Epoch [23/100], Loss: 0.0020
Epoch [23/100], Loss: 0.0063
Epoch [23/100], Loss: 0.0213
Epoch [23/100], Loss: 0.0202
Epoch [23/100], Loss: 0.0596
Epoch [23/100], Loss: 1.9918
Epoch [23/100], Loss: 0.0044
Epoch [23/100], Loss: 0.0009
Epoch [23/100], Loss: 0.1667
Epoch [23/100], Loss: 0.7462
Epoch [23/100], Loss: 0.1543
Epoch [23/100], Loss: 0.4369
Epoch [23/100], Loss: 0.0001
Epoch [23/100], Loss: 0.0111
Epoch [23/100], Loss: 0.0709
Epoch [23/100], Loss: 0.0104
Epoch [23/100], Loss: 0.0344
Epoch [23/100], Loss: 0.0368
Epoch [23/100], Loss: 0.0005
Epoch [23/100], Loss: 0.0002
Epoch [23/100], Loss: 0.0225
Epoch [23/100], Loss: 0.0772
Epoch [23/100], Loss: 0.0001
Epoch [23/100], Loss: 0.1181
Epoch [23/100], Loss: 0.0238
Epoch [23/100]

Epoch [24/100], Loss: 0.0056
Epoch [24/100], Loss: 0.1314
Epoch [24/100], Loss: 0.1157
Epoch [24/100], Loss: 0.0028
Epoch [24/100], Loss: 0.0955
Epoch [24/100], Loss: 0.4903
Epoch [24/100], Loss: 0.1492
Epoch [24/100], Loss: 0.5088
Epoch [24/100], Loss: 0.0024
Epoch [24/100], Loss: 0.4227
Epoch [24/100], Loss: 0.0216
Epoch [24/100], Loss: 0.3143
Epoch [24/100], Loss: 0.0100
Epoch [24/100], Loss: 0.0875
Epoch [24/100], Loss: 0.1128
Epoch [24/100], Loss: 0.0901
Epoch [24/100], Loss: 0.0029
Epoch [24/100], Loss: 0.6290
Epoch [24/100], Loss: 4.7733
Epoch [24/100], Loss: 0.2593
Epoch [24/100], Loss: 0.0814
Epoch [24/100], Loss: 0.0186
Epoch [24/100], Loss: 0.0006
Epoch [24/100], Loss: 0.1394
Epoch [24/100], Loss: 0.0418
Epoch [24/100], Loss: 4.4427
Epoch [24/100], Loss: 1.2036
Epoch [24/100], Loss: 0.0322
Epoch [24/100], Loss: 0.0036
Epoch [24/100], Loss: 0.0017
Epoch [24/100], Loss: 1.2231
Epoch [24/100], Loss: 0.0023
Epoch [24/100], Loss: 0.0031
Epoch [24/100], Loss: 0.2740
Epoch [24/100]

Epoch [24/100], Loss: 0.0014
Epoch [24/100], Loss: 0.0099
Epoch [24/100], Loss: 0.0212
Epoch [24/100], Loss: 0.0811
Epoch [24/100], Loss: 0.0730
Epoch [24/100], Loss: 0.0935
Epoch [24/100], Loss: 0.0242
Epoch [24/100], Loss: 0.0138
Epoch [24/100], Loss: 0.0012
Epoch [24/100], Loss: 0.0702
Epoch [24/100], Loss: 0.1017
Epoch [24/100], Loss: 0.0359
Epoch [24/100], Loss: 0.0049
Epoch [24/100], Loss: 0.1377
Epoch [24/100], Loss: 0.0031
Epoch [24/100], Loss: 0.0532
Epoch [24/100], Loss: 0.1189
Epoch [24/100], Loss: 0.3465
Epoch [24/100], Loss: 0.0396
Epoch [24/100], Loss: 0.0125
Epoch [24/100], Loss: 0.0323
Epoch [24/100], Loss: 1.1070
Epoch [24/100], Loss: 0.1112
Epoch [24/100], Loss: 0.0245
Epoch [24/100], Loss: 0.7809
Epoch [24/100], Loss: 0.0754
Epoch [24/100], Loss: 0.0221
Epoch [24/100], Loss: 0.0719
Epoch [24/100], Loss: 2.9951
Epoch [24/100], Loss: 9.4388
Epoch [24/100], Loss: 0.0163
Epoch [24/100], Loss: 0.1530
Epoch [24/100], Loss: 0.1688
Epoch [24/100], Loss: 0.0228
Epoch [24/100]

KeyboardInterrupt: 

In [61]:
print(losses)

with open('losses.txt', 'a') as f:
    for item in losses:
        f.write("%s\n" % item)

[tensor([-3.1967]), tensor([-3.5724]), tensor([-3.4067]), tensor([-3.3865]), tensor([-3.7676])]
[-5.28, -6.88, -5.86, -6.17, -6.64]


In [ ]:
# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')